# **Limpieza de datos en Pandas: Explicado con ejemplos**

*Eliminar o transformar datos erróneos e irrelevantes*: Estos representan información fuera de contexto en nuestra base de datos, por ejemplo tener la variable precio de venta en números negativos.

Vamos a aprender a realizar la limpieza de un dataFrame con un caso prático: vamos a utilizar un DataSet que analiza los ataquetas de tiburones por paises.

In [ ]:
import pandas as pd

#desde csv
ataques = pd.read_excel("shark attack.xlsx")
ataques.head()



<h3>Paso 1: Observaciones iniciales:</h3>

<ol><li><strong>Dimensiones</strong></li><li><strong>Nombres de columnas</strong></li><li><strong>Tipos de datos</strong>

In [ ]:
#Vemos las dimesiones, sus columnas y sus tipos
print(ataques.shape)
print(ataques.columns)
print(ataques.dtypes)

#¿Cuáles son los únicos que no contienen texto?

<h3>Paso 2: Resumen general de los datos:</h3>

In [ ]:
# Estadísticas básicas para columnas numéricas
print(ataques.describe())


<h3>Paso 3: Limpieza de datos:</h3>

Para preparar la limpieza de datos,vamos a realizar una serie de comprobaciones:

<ol><li><strong>
Eliminar columnas innecesarias</strong>:<strong>Número de columnas y nombres</strong>: Hay 23 columnas. Algunas parecen redundantes, como <code>Case Number</code>, <code>Case Number.1</code>, y <code>Case Number.2</code>. Columnas como Unnamed: 21, Unnamed: 22, y otras redundantes pueden ser eliminadas.</li><li><strong>Valores nulos</strong>: Existen columnas con valores nulos, como <code>Age</code> y <code>Species</code>.</li><li><strong>Nombres poco claros</strong>: Algunas columnas como <code>Unnamed: 21</code> y <code>Unnamed: 22</code> parecen no tener información útil.</li><li><strong>Tipos de datos</strong>: Algunos parecen inconsistentes, como <code>Age</code> que podría estar mezclando números y texto.</li></ol>



**1. Análisis de información de columnas. ¿Sobrán columnas ? , ¿ Es información válida ?**

Podemos usar el método isnull().sum() para contar los valores nulos y nunique() para verificar cuántos valores únicos tiene cada columna:

In [ ]:
# Contar valores nulos en cada columna
print(ataques.isnull().sum())

In [ ]:
# Contar valores únicos en cada columna
print(ataques.nunique())

In [ ]:
#Comprobar correlaciones o patrones en columnas redundantes
# Si las columnas parecen ser duplicados de otras, puedes verificar si son iguales:
# Comparar columnas aparentemente duplicadas
print(ataques['Case Number'].equals(ataques['Case Number.1']))
print(ataques['Case Number'].equals(ataques['Case Number.2']))



In [ ]:
valores_distintos = set(ataques['Case Number'])-(set(ataques['Case Number.1']))
print(valores_distintos)

In [ ]:
# Usar la función set de Python para encontrar valores distintos
valores_distintos = set(ataques['href'])-(set(ataques['href formula']))
len(valores_distintos)

**Eliminamos columnas repetidas. Drop**

Una vez que hemos analizado y observado las columnas, tomamos decisiones.

<div class="markdown prose w-full break-words dark:prose-invert light"><hr><h4><strong> Cuándo eliminar una columna repetida</strong></h4><h4>Reglas comunes:</h4><ul><li><p><strong>Similitud alta (95-100%)</strong>:</p><ul><li>Si las columnas son casi idénticas (por ejemplo, más del 95% iguales), una de ellas puede ser eliminada por redundancia.</li><li>Ejemplo: <code>Case Number</code> y <code>Case Number.1</code> podrían ser redundantes.</li></ul></li><li><p><strong>Diferencias menores (5-10%)</strong>:</p><ul><li>Si hay pequeñas diferencias pero el significado de la columna es idéntico (por ejemplo, errores tipográficos o formatos distintos), puedes mantener una sola versión y corregir los valores diferentes.</li></ul></li><li><p><strong>Diferencias significativas (&gt;20%)</strong>:</p><ul><li>Si más del 20% de los valores son diferentes, probablemente ambas columnas contengan información valiosa y no debas eliminarlas.</li></ul></li></ul><hr><h4><strong>Decisiones basadas en el contexto</strong></h4><ul><li><strong>Columnas derivadas automáticamente</strong>: Por ejemplo, columnas generadas durante una exportación de datos (<code>Unnamed</code>, <code>href formula</code>) suelen ser candidatas para eliminación inmediata.</li><li><strong>Redundancia conceptual</strong>: Si dos columnas representan lo mismo (como códigos y descripciones que son equivalentes), mantén la que sea más útil para el análisis.</li><li><strong>Información única</strong>: Si una columna aparentemente redundante contiene información única en ciertas filas, evalúa si puedes combinar ambas columnas.</li></ul><hr></p></div>

In [ ]:
# Calcular similitud entre las columnas
# [1, 1, 0, 1]  # True -> 1, False -> 0
similaridad = (ataques['href'] == ataques['href formula']).mean() * 100
print(f"Porcentaje de valores iguales: {similaridad:0.2f}%")


In [ ]:
similaridad = (ataques['Case Number'] == ataques['Case Number.1']).mean() * 100
print(f"Porcentaje de valores iguales: {similaridad:0.2f}%")



In [ ]:
# Eliminar columnas innecesarias
ataques = ataques.drop(columns=['Unnamed: 21', 'Unnamed: 22', 'Case Number.1', 'Case Number.2', 'href formula'])


**2. Detección de datos nulos**

La función *pandas.isnull()* devuelve una estructura con las mismas dimensiones sustituyendo cada valor por el booleano True si el correspondiente elemento es un valor nulo, y por el booleano False en caso contrario.

In [ ]:
null_cols_values = ataques.isnull()


In [ ]:
null_cols = ataques.isnull().sum()
null_cols

<hr><h4><p><strong>Lidiar con valores nulos: Dependiendo de la columna, se pueden rellenar valores nulos, eliminarlos o mantenerlos</strong></h4>

El método *fillna()* permite sustituir los valores nulos de una estructura pandas por otro valor según ciertos criterios.

Pueden sustituirse por un valor concreto o bien puede utilizarse el anterior o posterior valor no nulo.

Para Age: Rellenar con la mediana o un valor específico si es relevante.
Para Species: Rellenar con "Desconocido" si es categórico.



In [ ]:
# Rellenar nulos con la mediana
ataques.fillna({'Age': ataques['Age'].median()}, inplace=True)
print(null_cols)



In [ ]:
# Convertir edades a numéricom y pone a nulos los que no se pueden cambiar
ataques['Age'] = pd.to_numeric(ataques['Age'], errors='coerce') 

In [ ]:
ataques.fillna({'species':'Unknown'}, inplace=True)           # Rellenar valores nulos con "Unknown"

In [ ]:
null_cols = ataques.isnull().sum()
null_cols

<hr><h4><p><strong>Seguiríamos investigando y trabajando con el resto de columnas disponibles...</strong></h4><hr>

Analizamos ahora la columna Sex,antes de nada, observamos que el nombre de la columna tiene un espacio
set(ataques['Sex '])

¿Qué podemos hacer? Con **str.strip()** eliminanos los caracteres de espacio en blanco al principio y al final de cada cadena, dejando el contenido de la cadena intacto. También puede recibir un argumento opcional que te permite especificar qué caracteres deseas eliminar en lugar de los espacios en blanco.

In [ ]:

ataques.columns = ataques.columns.str.strip()
ataques.columns

In [ ]:
#Ahora si observamos los datos:
set(ataques['Sex'])



In [ ]:
# Analizamos ahora la columna Fatal (Y/N),con la operación set mostramos el conjunto de valores que tenemos
set(ataques['Fatal (Y/N)'])

In [ ]:
len(set(ataques['Fatal (Y/N)']))


Lo unificamos paso a paso, objetivo, conseguir este set:
{'Y', 'N', 'U'}

In [ ]:
# Los unificamos paso a paso
#quitamos espacios vacios
ataques['Fatal (Y/N)'] = ataques['Fatal (Y/N)'].str.strip()
#reemplazamos los nulos por U
ataques['Fatal (Y/N)'] = ataques['Fatal (Y/N)'].fillna('U')
#reemplazamos los nulos por U
ataques['Fatal (Y/N)'] = ataques['Fatal (Y/N)'].str.
......

In [ ]:
#Cambiar nombre 'Fatal (Y/N)' a 'Fatal (Y/N/U)'
ataques.rename(columns={'Fatal (Y/N)': 'Fatal (Y/N/U)'}, inplace=True)

# EJERCICIO:

1. Limpiamos columna Sex
2. Columna Country
3. Reemplazar los valores nulos de la columna 'Species' con 'Unknown'
4. Guarda el resultado en un csv nuevo

In [ ]:
# Limpiamos la columna Sex
ataques['Sex']